In [80]:
import pandas as pd

In [81]:
def code_prefix(x):
    if len(str(x)) == 3:
        return 'ms-drg'
    elif len(str(x)) == 5:
        return 'hcpcs_cpt'
    else:
        return 'none'
    
def payer_category(x):
    payers = {
        'PRICING_AMT': 'gross',
        'Self Pay/Uninsured': 'cash',
        'Pricing Max': 'max',
        'Pricing Min': 'min'
    }

    try:
        return payers[x]
    except KeyError:
        return 'payer'

In [82]:
file = '59-1943502_ShandsTeachingHospitalandClinics_standardcharges (3).csv'
df = pd.read_csv(file, dtype=str)

In [83]:
df.rename(columns={
    'PROCEDURE': 'internal_code',
    'UNIT': 'ndc_quantity_desc',
    'SUF EAP DESC': 'description',
    'CODE': 'code_orig'
},
    inplace=True)

In [84]:
df.drop('Eff Date', axis=1, inplace=True)

In [85]:
cols = df.columns.tolist()
id_vars = cols[:4]
val_vars = cols[4:]

df = pd.melt(df, id_vars=id_vars, value_vars=val_vars, value_name='rate', var_name='payer_orig')

In [86]:
df['code_prefix'] = df['code_orig'].apply(code_prefix)
df['payer_category'] = df['payer_orig'].apply(payer_category)

In [87]:
df.loc[df['ndc_quantity_desc'].isin(['1 each', 'Ea', 'DISCHARGE']), 'ndc_quantity_desc'] = pd.NA

In [88]:
df['rate'] = df['rate'].str.replace('$', '').str.replace(',', '')

C:\Users\adria\AppData\Local\Temp\ipykernel_167248\505826267.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['rate'] = df['rate'].str.replace('$', '').str.replace(',', '')


In [89]:
df['file_last_updated'] = '2023-01-01'
df['hospital_ccn'] = '100113'
df['hospital_ein'] = '59-1943502'
df['url'] = 'https://ufhealth.org/sites/default/files/media/financial-assistance/59-1943502_ShandsTeachingHospitalandClinics_standardcharges%20%283%29.csv'
df['filename'] = file

In [90]:
df['payer_name'] = df['payer_orig']
df['code'] = df['code_orig']

In [91]:
df['payer_orig'] = df['payer_orig'].str.strip()
df['payer_name'] = df['payer_name'].str.strip()
df['description'] = df['description'].str.strip()
df['code'] = df['code'].str.strip()
df['code_orig'] = df['code_orig'].str.strip()
df['rate'] = df['rate'].str.strip()

# This has to be under the strip because a lot of blank codes are spaces
df['code'].fillna('na', inplace=True)
df.loc[df['code'].isnull() | (df['code'] == ''), 'code'] = 'na'

In [92]:
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]

df = df[cols]

In [93]:
df['rate'] = df['rate'].astype(str).str.strip()
# df = df[~df['rate'].isin(['Bundled', 'Markup'])]
df = df[pd.to_numeric(df['rate'], errors='coerce').notnull()]

# df.dropna(subset=['rate'], inplace=True)

In [94]:
df.to_csv('shands.csv', index=False)